#### Data_Hälso_förjupning.

In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dataclasses import dataclass
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression


np.random.seed(42)


In [88]:
def visa_statistik(df):
    kolumner = ["age", "systolic_pb", "cholesterol", "BMI" ]
    for kol in kolumner:
        print(f"{kol}: medel={df[kol].mean():.f}, median={df[kol].median():.f}")

def skapa_grafer(df):
    fig, axs = plt.Subplots(2, 2, figsize=(12, 8))

    num_kol = ["age", "systolic_pb", "cholesterol", "BMI" ]
    sns.heatmap(df[num_kol].corr(),annot=True, ax=axs[0,0])

    axs[0,1].scatter(df["age"], df["systolic_pb"], alpha=0.5)
    axs[0,1].set_title("Ålder vs Blodtryck")
    axs[0,1].boxplot([df[df["sex"]=="M"]["systolick_pb"],
                    df[df["sex"]=="F"]["systolic_pb"]],
                    labels=["Män", "Kvinnor"])
    
    sjukdom_per_kön = df.groupby("sex"["disease"].mean() * 100)
    axs[1,1].bar("sjuk_per_kön.index, sjuk_per_kön.values")
    
    plt.tight_layout()
    plt.show()
    

<>:18: SyntaxWarning: str indices must be integers or slices, not str; perhaps you missed a comma?
<>:18: SyntaxWarning: str indices must be integers or slices, not str; perhaps you missed a comma?
C:\Users\zanya\AppData\Local\Temp\ipykernel_20964\3324585049.py:18: SyntaxWarning: str indices must be integers or slices, not str; perhaps you missed a comma?
  sjukdom_per_kön = df.groupby("sex"["disease"].mean() * 100)


#### Klasser

In [89]:
class HealsAnalyz:

    def __init__(self, data):
        self.data = data

    def sammanfattning(self):
        return{
            "antal": len(self.data),
            "antal_sjuk": self.data["disease"].mean() * 100,
            "medelålder": self.data["age"].mean()
        }
    
    def regression(self, x_kol, y_kol):
        x = self.data[[x_kol]] if isinstance(x_kol, str) else self.data[x_kol]
        y =self.data[y_kol]

        
        model = LinearRegression()
        model.fit(x, y)

        return {
            "r2": model.score(x, y),
            "koeff": model.coef_,
            "intercept": model.intercept_
        }
   
analysator = HealsAnalyz(df)
print("sammanfattning:")
for nyckel, värde in analysator.sammanfattning().items():
        print(f"{nyckel}: {värde}")




sammanfattning:
antal: 800
antal_sjuk: 5.875
medelålder: 49.42625
